# Creating rasters of annual inundation metrics using Landsat

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with `DEA Sandbox` environment
* **Products used:** 
[ga_s2am_ard_3](https://explorer.dea.ga.gov.au/products/ga_s2am_ard_3)  [ga_s2bm_ard_3](https://explorer.dea.ga.gov.au/products/ga_s2bm_ard_3)

### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

from odc.stac import configure_s3_access, load
import matplotlib.pyplot as plt
from pystac_client import Client as PystacClient
import numpy as np
import geopandas as gpd
import odc.geo.xr
import xarray as xr
from pprint import pprint

#from dea_tools.bandindices import calculate_indices
#from dea_tools.dask import create_local_dask_cluster
#from dea_tools.datahandling import load_ard
#from dea_tools.plotting import display_map, xr_animation
from odc.geo.geom import Geometry

from IPython.display import Image
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

In [1]:
from eo_insights.stac_configuration import de_australia_stac_config
from eo_insights.raster_base import RasterBase, QueryParams, LoadParams
from odc.algo import erase_bad, mask_cleanup

ModuleNotFoundError: No module named 'odc.algo'

### Connect to the datacube

Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [24]:
catalog = "https://explorer.dea.ga.gov.au/stac"

stac_client = PystacClient.open(catalog)

configure_s3_access(
    cloud_defaults=True,
    aws_unsigned=True,
)

## Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Australia, see the [Dask notebook](../Beginners_guide/09_Parallel_processing_with_Dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in JupyterLab* section of the [Dask notebook](../Beginners_guide/09_Parallel_processing_with_Dask.ipynb).

To activate Dask, set up the local computing cluster using the cell below.

In [7]:
from dask.distributed import Client as DaskClient

dask_client = DaskClient()
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.62 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40903,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.62 GiB
Comm: tcp://127.0.0.1:44775,Total threads: 1
Dashboard: http://127.0.0.1:38031/status,Memory: 3.90 GiB
Nanny: tcp://127.0.0.1:38797,


## Load cloud-masked satellite data

The code below will create a query dictionary for our region of interest, and then load Sentinel-2 satellite data.
For more information on loading data, see the [Loading data notebook](../Beginners_guide/04_Loading_data.ipynb).

In [26]:
date_query = ['1990-07','1991-06']

collections_query = ['ga_ls5t_ard_3']


In [27]:
items = stac_client.search(
    collections = collections_query,
    bbox= [145.910476061,-29.153361653,148.745019431,-30.128112653],
    datetime = date_query,    
).item_collection()

print(f"Found {len(items)} items")


#bbox south [145.910476061,-29.153361653,148.745019431,-30.128112653]
#bbox north [146.963236949,-27.723499515,148.806899103,-29.169637680]

Found 175 items


In [30]:
ds = load(
    items,
    bands=['nbart_red', 'nbart_green', 'nbart_nir',
                     'nbart_swir_1', 'nbart_swir_2', 'oa_fmask'],
    crs="utm",
    chunks={"time": -1, "x": 400, "y": 400},
    resolution=30,
    groupby="solar_day",
    bbox= [145.910476061,-29.153361653,148.745019431,-30.128112653],
)


ds_masked = ds.apply_mask("fmask", data_inplace=False, mask_inplace=False)

AttributeError: 'Dataset' object has no attribute 'apply_mask'

## Calculate the Fisher water index, binarise water and edit date type

IMPORTANT: change the reference date accordingly

In [ ]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
ds['fwi'] = 1.7204 + 171*(ds.nbart_green/10000) + 3*(ds.nbart_red/10000) - 70*(ds.nbart_nir/10000) - 45*(ds.nbart_swir_2/10000)-71*(ds.nbart_swir_3/10000)

#Binarise water
ds['water'] = (ds['fwi'] >= -12.67).astype(np.int8)

# Define the reference period (start date)
reference_date = np.datetime64('2016-07-01')

# Transform date values to the number of days since the reference period
days_since_reference = (ds.time.values - reference_date).astype('timedelta64[D]').astype(int)

ds.coords["time"] = days_since_reference

ds

<xarray.Dataset> Size: 5GB
Dimensions:       (time: 12, y: 3224, x: 4283)
Coordinates:
  * y             (y) float64 26kB -3.316e+06 -3.316e+06 ... -3.349e+06
  * x             (x) float64 34kB 1.434e+06 1.434e+06 ... 1.477e+06 1.477e+06
    spatial_ref   int32 4B 3577
  * time          (time) int64 96B 44 84 124 144 154 164 184 234 244 284 324 364
Data variables:
    nbart_swir_3  (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    nbart_green   (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    nbart_red     (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    nbart_nir_1   (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    oa_fmask      (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    fwi           (time, y, x) float32 663MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
    water         (time, y, x) int8 166MB dask.array<chunksize=(12, 500, 500), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

### Functions to obtain longest inundation period and date of start. 

Creates 2 arrays which we then combine into a dataset to write into a tiff

In [11]:
def time_difference_slice(arr, times):
    diff = np.diff(arr, axis = 0)
    starts = np.where(diff == 1)[0] + 1
    ends = np.where(diff == -1)[0] + 1

    if arr[0]:
        starts = np.insert(starts, 0, 0)
    if arr[-1]:
        ends = np.append(ends, len(diff))

    if len(starts) == 0 or len(ends) == 0:
        return 0  # Return default value if there are no valid starts or ends

    lengths = ends - starts
    if lengths.size == 0:
        return 0  # Return default value if no consecutive 1s are found

    max_len = np.max(lengths)
    idx = np.argmax(lengths)

    start_idx = starts[idx]
    end_idx = ends[idx] - 1

    start_time = times[start_idx]
    end_time = times[end_idx]

    if (end_time + 1) > start_time:
        time_difference = (end_time + 1) - start_time
    else:
        time_difference = 0
    

    return time_difference

def start_time_slice(arr, times):
    diff = np.diff(arr, axis = 0)
    starts = np.where(diff == 1)[0] + 1
    ends = np.where(diff == -1)[0] + 1

    if arr[0]:
        starts = np.insert(starts, 0, 0)
    if arr[-1]:
        ends = np.append(ends, len(diff))

    if len(starts) == 0 or len(ends) == 0:
        return np.nan  # Return default value if there are no valid starts or ends

    lengths = ends - starts
    if lengths.size == 0:
        return np.nan  # Return default value if no consecutive 1s are found

    idx = np.argmax(lengths)
    start_idx = starts[idx]

    end_idx = ends[idx] - 1

    start_time = times[start_idx]
    end_time = times[end_idx]    

    if end_time > start_time:
        start_time = times[start_idx]
    else:
        start_time = np.nan

        
    return start_time

length_array = xr.apply_ufunc(
    time_difference_slice,
    ds.water,
    ds.time,
    input_core_dims=[['time'], ['time']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float]
)

start_time_array = xr.apply_ufunc(
    start_time_slice,
    ds.water,
    ds.time,
    input_core_dims=[['time'], ['time']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float]
)


dsyearlywaterdyn = xr.Dataset(
    {
        "length": (["y", "x"], length_array.data),
        "start_time": (["y", "x"], start_time_array.data),
    },
    coords={"x": ds.water.x, "y": ds.water.y}
) 

### Write the dataset into a tiff file. 

Output will save in DEA Sandbox directory from where it can be downloaded

In [8]:
%%time
# Write multi-band GeoTIFF to a location
from datacube.utils.cog import write_cog

dsyearlywaterdyn_array = dsyearlywaterdyn.to_array()


file = write_cog(dsyearlywaterdyn_array,
          fname='narran_annual_watermetrics_1617.tif',
          overwrite=True)


file.compute()

CPU times: user 6min 21s, sys: 16.1 s, total: 6min 37s
Wall time: 47min 31s


PosixPath('narran_annual_watermetrics_1718.tif')

When done close the Dask client to release reources

In [17]:
client.close()

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Discord chat](https://discord.com/invite/4hhBQVas5U) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [GitHub](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** July 2024

**Compatible datacube version:**

In [22]:
print(datacube.__version__)

1.8.18
